In [1]:
#IMPORTAR PACKAGES y LIBRARYS

#Librerías útiles
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm
from pmdarima.utils import diff_inv
from datetime import datetime, timedelta
import itertools
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
#otros
import warnings
warnings.filterwarnings("ignore")

#Modelos de Pruebas Estadísticas


import statsmodels.tsa.stattools as ts

#Modelos predictivos estadísticos
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.diagnostics import performance_metrics
from prophet.diagnostics import cross_validation
from prophet.plot import plot_cross_validation_metric


#Configuraciones
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#DICCIONARIOS, OBJETOS
dic_mes = {
        'Enero': 1,
        'Febrero': 2,
        'Marzo': 3,
        'Abril':4,
         'Mayo':5,
         'Junio':6,
         'Julio':7,   
         'Agosto':8,
         'Setiembre':9,
         'Octubre':10,
         'Noviembre':11,
         'Diciembre':12
        }


In [3]:
#PREPARACIÓN_DATA

df= pd.read_excel("C:/Users/Usuario/OneDrive - prediqtdata.com/Desktop/Renato/Python/script atenea/consolidado_moderno.xlsx"
,sheet_name="Detergentes")
df.replace(dic_mes, inplace=True)
df['Quarter'] = df['MES'].apply(lambda x: 'Quarter1' if x in [1, 2, 3] else 'Quarter2' if x in [4, 5, 6] else 'Quarter3' if x in [7, 8, 9] else 'Quarter4')
df['Month_in_Quarter'] = df['MES'].apply(lambda x: 1 if x % 3 == 1 else 2 if x % 3 == 2 else 3)
df["PERIODO"]= df["AÑO"].astype(str)+"-"+df["MES"].astype(str)


df['PERIODO'] = pd.to_datetime(df['PERIODO']) 
df = df.rename(columns={'PERIODO': 'ds',
                        'VENTA SELL IN': 'y'})


cols=['PERIODO','AÑO','MES','Quarter','Month_in_Quarter','OFICINA DE VENTAS'
      ,'VENTA SELL IN', 'VENTA SELL OUT TOTAL'
      , 'SHARE VOLUMEN ALICORP', 'SHARE VOLUMEN MERCADO'
      ,'SHARE % ALICORP','SHARE % MERCADO']

cols_data =['ds','y'
      #, 'VENTA SELL OUT TOTAL'
      #, 'SHARE VOLUMEN ALICORP'
      #,'SHARE VOLUMEN MERCADO'
      #,'SHARE % ALICORP'
      #,'SHARE % MERCADO'
      ]


df=df[cols_data]

cutoffs = pd.date_range(start='2021-02-01', end='2023-06-01', freq='1MS') -timedelta(days=1)
horizon_var = "10 days"


In [4]:
#Parameter tuning(hayando mejores parametros)

training_data = df

param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    model = Prophet(**params).fit(df)  # Fit model with given params
    df_cv = cross_validation(model,horizon=horizon_var,cutoffs=cutoffs)
    df_p = performance_metrics(df_cv)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

"training_data = df\n\nparam_grid = {  \n    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],\n    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],\n}\n\n# Generate all combinations of parameters\nall_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]\nrmses = []  # Store the RMSEs for each params here\n\n# Use cross validation to evaluate all parameters\nfor params in all_params:\n    model = Prophet(**params).fit(df)  # Fit model with given params\n    df_cv = cross_validation(model,horizon=horizon_var,cutoffs=cutoffs)\n    df_p = performance_metrics(df_cv)\n    rmses.append(df_p['rmse'].values[0])\n\n# Find the best parameters\ntuning_results = pd.DataFrame(all_params)\ntuning_results['rmse'] = rmses\nprint(tuning_results)"

In [5]:
#Mostrar mejores parametros
best_params = all_params[np.argmin(rmses)]
best_params=list(best_params.items())
#best_params

In [6]:
#Prophet(changepoint_prior_scale=best_params[0][1],
#               seasonality_prior_scale=best_params[1][1])

In [7]:
#PRIMER MODELO CON HIPERPARAMETROS TUNNED

model_1 = Prophet(changepoint_prior_scale=best_params[0][1],
               seasonality_prior_scale=best_params[1][1]).fit(df)
future = model_1.make_future_dataframe(periods=6,freq="MS")
forecast = model_1.predict(future)
#fig1 = model_1.plot(forecast)
#fig2 = model_1.plot_components(forecast)
plot_plotly(model_1, forecast)
#plot_components_plotly(model_1, forecast)
df_cv = cross_validation(model_1,horizon=horizon_var,cutoffs=cutoffs)
df_p = performance_metrics(df_cv)

df_p

10:24:57 - cmdstanpy - INFO - Chain [1] start processing
10:24:57 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/29 [00:00<?, ?it/s]

10:24:57 - cmdstanpy - INFO - Chain [1] start processing
10:24:57 - cmdstanpy - INFO - Chain [1] done processing
10:24:58 - cmdstanpy - INFO - Chain [1] start processing
10:24:58 - cmdstanpy - INFO - Chain [1] done processing
10:24:58 - cmdstanpy - INFO - Chain [1] start processing
10:24:58 - cmdstanpy - INFO - Chain [1] done processing
10:24:58 - cmdstanpy - INFO - Chain [1] start processing
10:24:58 - cmdstanpy - INFO - Chain [1] done processing
10:24:58 - cmdstanpy - INFO - Chain [1] start processing
10:25:01 - cmdstanpy - INFO - Chain [1] done processing
10:25:02 - cmdstanpy - INFO - Chain [1] start processing
10:25:02 - cmdstanpy - INFO - Chain [1] done processing
10:25:02 - cmdstanpy - INFO - Chain [1] start processing
10:25:02 - cmdstanpy - INFO - Chain [1] done processing
10:25:02 - cmdstanpy - INFO - Chain [1] start processing
10:25:20 - cmdstanpy - INFO - Chain [1] done processing
10:25:20 - cmdstanpy - INFO - Chain [1] start processing
10:25:27 - cmdstanpy - INFO - Chain [1]

,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,1 days,158636.637607,398.292151,271.83091,0.13534,0.095379,0.113496,0.896552


In [8]:
#ENCONTRAR OUTLIERS
performance = pd.merge(df, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on='ds')
performance_MAE = mean_absolute_error(performance['y'], performance['yhat'])
print(f'The MAE for the model is {performance_MAE}')

# Check MAPE value
performance_MAPE = mean_absolute_percentage_error(performance['y'], performance['yhat'])
print(f'The MAPE for the model is {performance_MAPE}')
# Create an anomaly indicator
performance['anomaly'] = performance.apply(lambda rows: 1 if ((rows.y<rows.yhat_lower*0.75)|(rows.y>rows.yhat_upper*1.25)) else 0, axis = 1)

# Check the number of anomalies
performance['anomaly'].value_counts()
anomalies = performance[performance['anomaly']==1].sort_values(by='ds')
performance['y'] = performance.apply(lambda rows: np.NaN if rows.anomaly == 1 else rows.y, axis = 1)
window_size = 3
performance['y']=performance['y'].fillna(performance['y'].rolling(window_size+1, min_periods=1).mean())
df_2= performance[["ds","y"]]



The MAE for the model is 260.11607521047716
The MAPE for the model is 0.11851087530710423


anomaly
0    53
1     2
Name: count, dtype: int64

In [9]:
#MODELO SIN OUTLIERS
model_2 = Prophet(changepoint_prior_scale=0.001,
                seasonality_prior_scale=0.01).fit(df_2)  
future = model_2.make_future_dataframe(periods=6,freq="MS")
forecast = model_2.predict(future)
#fig1 = model_2.plot(forecast)
#fig2 = model_2.plot_components(forecast)
#plot_plotly(model_2, forecast)

df_cv = cross_validation(model_2,horizon=horizon_var,cutoffs=cutoffs)
df_p = performance_metrics(df_cv)
df_p

10:25:57 - cmdstanpy - INFO - Chain [1] start processing
10:25:58 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/29 [00:00<?, ?it/s]

10:25:58 - cmdstanpy - INFO - Chain [1] start processing
10:25:58 - cmdstanpy - INFO - Chain [1] done processing
10:25:58 - cmdstanpy - INFO - Chain [1] start processing
10:25:59 - cmdstanpy - INFO - Chain [1] done processing
10:25:59 - cmdstanpy - INFO - Chain [1] start processing
10:25:59 - cmdstanpy - INFO - Chain [1] done processing
10:25:59 - cmdstanpy - INFO - Chain [1] start processing
10:25:59 - cmdstanpy - INFO - Chain [1] done processing
10:25:59 - cmdstanpy - INFO - Chain [1] start processing
10:25:59 - cmdstanpy - INFO - Chain [1] done processing
10:26:00 - cmdstanpy - INFO - Chain [1] start processing
10:26:00 - cmdstanpy - INFO - Chain [1] done processing
10:26:00 - cmdstanpy - INFO - Chain [1] start processing
10:26:00 - cmdstanpy - INFO - Chain [1] done processing
10:26:00 - cmdstanpy - INFO - Chain [1] start processing
10:26:00 - cmdstanpy - INFO - Chain [1] done processing
10:26:00 - cmdstanpy - INFO - Chain [1] start processing
10:26:01 - cmdstanpy - INFO - Chain [1]

,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,1 days,115320.504698,339.588729,279.358852,0.117504,0.080983,0.108635,0.655172
